In [1]:
from jax import numpy as jnp
import transformers
from transformers import (
    AutoTokenizer, 
    AutoModelForMaskedLM, 
    BertForMaskedLM, 
    BertTokenizer, 
    BertTokenizerFast, 
    BertEmbeddings,
    BfBertEmbeddings,
    BertConfig,
)



/home/kevin/miniconda3/envs/jax-hf/lib/python3.9/site-packages/tqdm/auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Establish data
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")
text = ["hello I want to eat some [MASK] meat today. It's thanksgiving [MASK] all!", "yo yo what's up"]

# tokenize text and pass into model
tokens = tokenizer(text, return_tensors="pt", padding=True)
input_ids = tokens["input_ids"]
print(input_ids)

tensor([[  101,  7592,  1045,  2215,  2000,  4521,  2070,   103,  6240,  2651,
          1012,  2009,  1005,  1055, 15060,   103,  2035,   999,   102],
        [  101, 10930, 10930,  2054,  1005,  1055,  2039,   102,     0,     0,
             0,     0,     0,     0,     0,     0,     0,     0,     0]])


In [6]:
# Create BfBertEmbeddings and BertEmbeddings
config = BertConfig.from_pretrained(pretrained_model_name_or_path="/home/kevin/code/rycolab/brunoflow/models/bert/config.json")
bf_embs = BfBertEmbeddings(config)
torch_embs = BertEmbeddings(config)

/home/kevin/code/rycolab/transformers/src/transformers/models/bert/modeling_bf_bert.py:174: UserWarning: Explicitly requested dtype <class 'jax.numpy.int64'> requested in zeros is not available, and will be truncated to dtype int32. To enable more dtypes, set the jax_enable_x64 configuration option or the JAX_ENABLE_X64 shell environment variable. See https://github.com/google/jax#current-gotchas for more.
  self.token_type_ids = jnp.zeros(self.position_ids.shape, dtype=jnp.int64) # todo is this 64 bit necessary?


In [9]:
# Compare output of BfBertEmbeddings and BertEmbeddings on the text
jax_input_ids = jnp.array(input_ids.numpy(), dtype=int)
print(bf_embs(input_ids=jax_input_ids).val)
print(torch_embs(input_ids=input_ids))

[[[-0.34472668  0.5307575  -0.6256574  ... -0.74307156 -0.7948409
    0.410834  ]
  [-0.12443667 -0.8550836  -0.33784032 ... -0.03916439 -1.4545143
    0.26670295]
  [-2.1477165  -0.90835226 -1.105538   ...  0.75945294 -1.8616817
   -0.16157249]
  ...
  [-1.8152208   0.4767846  -0.3528018  ... -0.03726343 -2.2972047
    0.14477256]
  [-0.9735971  -2.01189    -0.08289954 ... -0.51852256 -1.0732709
    0.9564046 ]
  [-1.0029035  -1.2219137   0.8737193  ... -0.09658846 -1.1882741
    0.27896053]]

 [[-0.34472668  0.5307575  -0.6256574  ... -0.74307156 -0.7948409
    0.410834  ]
  [-0.21959372 -1.3216001  -0.11712483 ... -0.14018822 -1.1034871
    0.9674713 ]
  [-2.029127   -1.2888647  -1.9155287  ...  0.5731998  -0.8514466
    0.7274356 ]
  ...
  [-1.2329972   0.89194095 -0.20066781 ... -0.42031294 -0.9897826
   -0.20450695]
  [-2.481228   -2.043665   -0.8651028  ... -0.5931333  -0.5373075
   -0.54956347]
  [-1.3868319  -0.63329226 -0.24705154 ... -0.3211176  -0.5438302
   -0.5995097 ]]]


In [10]:
be.position_ids
be.word_embeddings

Embedding(30522, 768, padding_idx=0)

In [2]:
model = BertForMaskedLM.from_pretrained("bert-base-uncased")
tokenizer = BertTokenizerFast.from_pretrained("bert-base-uncased")


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForMaskedLM: ['cls.seq_relationship.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [4]:
model._modules

OrderedDict([('bert',
              BertModel(
                (embeddings): BertEmbeddings(
                  (word_embeddings): Embedding(30522, 768, padding_idx=0)
                  (position_embeddings): Embedding(512, 768)
                  (token_type_embeddings): Embedding(2, 768)
                  (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
                  (dropout): Dropout(p=0.1, inplace=False)
                )
                (encoder): BertEncoder(
                  (layer): ModuleList(
                    (0): BertLayer(
                      (attention): BertAttention(
                        (self): BertSelfAttention(
                          (query): Linear(in_features=768, out_features=768, bias=True)
                          (key): Linear(in_features=768, out_features=768, bias=True)
                          (value): Linear(in_features=768, out_features=768, bias=True)
                          (dropout): Dropout(p=0.1, inplace=False)
    

In [27]:
s = ["hello woo isn't this so exciting?! I wanted to eat some chicken"]
tokens = tokenizer(s)
input_ids = tokens["input_ids"]


In [31]:
tokenizer.tokenize([s])

TypeError: TextEncodeInput must be Union[TextInputSequence, Tuple[InputSequence, InputSequence]]

In [26]:
tokenizer.get_vocab()

{'##pone': 29513,
 'sen': 12411,
 'disgrace': 29591,
 'cannon': 8854,
 'dal': 17488,
 'conclusions': 15306,
 'expense': 10961,
 '273': 25371,
 'temps': 29023,
 'ල': 1406,
 'smooth': 5744,
 '##ginal': 24965,
 'cm': 4642,
 'transylvania': 20816,
 '##mina': 22311,
 'comb': 22863,
 'finance': 5446,
 '##dson': 25112,
 'tatum': 25913,
 'canons': 21975,
 '镇': 1966,
 'ʔ': 1139,
 'anatomy': 13336,
 'abstracts': 29474,
 '[unused350]': 355,
 'italian': 3059,
 '##bad': 9024,
 'appointments': 14651,
 '##chfield': 22693,
 'given': 2445,
 'carolina': 3792,
 'lacey': 16355,
 'frenzy': 21517,
 'jana': 23341,
 'geometridae': 24687,
 'letterman': 26593,
 '家': 1825,
 'forged': 16158,
 '##∂': 30120,
 'violently': 14196,
 'specified': 9675,
 'harmful': 17631,
 'isolate': 27152,
 '241': 22343,
 '##mith': 19864,
 'officers': 3738,
 '##lan': 5802,
 'km': 2463,
 '##iating': 15370,
 '##yman': 17906,
 'campaign': 3049,
 'rapids': 12775,
 'seekers': 24071,
 '##anta': 26802,
 'according': 2429,
 'compliance': 12646

In [23]:
tokenizer.convert_ids_to_tokens(input_ids[0])

['[CLS]',
 'hello',
 'woo',
 'isn',
 "'",
 't',
 'this',
 'so',
 'exciting',
 '?',
 '!',
 'i',
 'wanted',
 'to',
 'eat',
 'some',
 'chicken',
 '[SEP]']